In [197]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max

In [198]:
spark = SparkSession\
    .builder\
    .appName('spark-lab')\
    .getOrCreate()

In [199]:
spark

In [200]:
url = "jdbc:postgresql://localhost:7760/railway"

properties = {
    "user": "postgres",
    "password": "",
    "driver": "org.postgresql.Driver"
}


1. Read last processed id 

In [201]:
pid =  spark.read.format("jdbc").option("url", url)\
    .option("driver", "org.postgresql.Driver")\
    .option("query", "select * from processed_id")\
    .option("user", properties['user'])\
    .option("password", properties['password'])\
    .load()
last_index = pid.select(max(pid['id'])).head()[0]
print(last_index)

6


2. Reading data using dbtable from first table

In [202]:
df1 = spark.read.format("jdbc").option("url", url) \
    .option("driver", "org.postgresql.Driver") \
    .option("query", f"select * from data where id > {last_index}") \
    .option("user", properties['user'])\
    .option("password", properties['password'])\
    .load()

In [203]:
df1.show()

+---+----+----+--------+
| id|name|city|pin-code|
+---+----+----+--------+
+---+----+----+--------+



3. Write data to another table

In [204]:
if df1.count() > 0 :
    df1.write.format("jdbc").option("url", url) \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", "hello") \
        .option("user", properties['user'])\
        .option("password", properties['password'])\
        .mode('append')\
        .save()
else:
    print('----------NO DATA TO UPDATE!----------')

----------NO DATA TO UPDATE!----------


4. Update the key in 

In [205]:

if df1.count() > 0:
    processedid=df1.select(max(df1.id).alias('id'))
    print('updating_table')
    processedid.write.format("jdbc").option("url", url) \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "processed_id") \
    .option("user", properties['user'])\
    .option("password", properties['password'])\
    .mode('overwrite')\
    .save()

else:
    print('----------NO DATA TO WRITE---------')
    

----------NO DATA TO WRITE---------
